<a href="https://colab.research.google.com/github/SunbirdAI/lamwo-electrification-project/blob/main/notebooks/village_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Village feature extraction

Extract different features at vilalge level from various types of vector, raster and CSV data for use in modeling.

In [1]:
%%capture
!pip install rasterstats

In [2]:
import pandas as pd
import geopandas as gpd
import rasterio
import json
import seaborn as sns
import warnings

from rasterstats import zonal_stats
warnings.filterwarnings('ignore')

Define path of the raw data. Use Google drive or unzip the data in `./raw/lamwo_data.zip`

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
gpath = '/content/drive/My Drive/Sunbird Local/GIZ/Phase 2/Technical/lamwo_data/'

Load the villages shapefile that is the basis of all the feature extraction

In [5]:
villages = gpd.read_file(gpath + "village boundaries/village_boundaries.shp")

Load GIS vector shapefiles for different types of data

In [6]:
candidate_minigrids = gpd.read_file(gpath + "candidate minigrids/candidate_minigrids.shp")
existing_minigrids = gpd.read_file(gpath + "existing minigrids/existing_minigrids.shp")
facilities = gpd.read_file(gpath + "facilities/facilities.shp")
grid_extension = gpd.read_file(gpath + "grid extension/grid_extension.shp")
existing_grid = gpd.read_file(gpath + "existing grid/existing_grid.shp")
buildings = gpd.read_file(gpath + "open buildings/open_buildings.shp")
roads = gpd.read_file(gpath + "roads/roads.shp")

Load raster layers

In [7]:
ndvi_raster = gpath + "ndvi/ndvi.tif"
wind_speed_raster = gpath + "wind/wind_speed.tif"
pvout_solar_radiation_raster = gpath + "solar radiation/pvout_solar_radiation.tif"

Load CSV files

In [8]:
land_use = pd.read_csv(gpath + "land use/land_use.csv")
protected_areas = pd.read_csv(gpath + "protected areas/protected_areas.csv")

Define different extraction helper functions

In [9]:
def count_points_in_polygon(points_gdf, polygon):
    return len(points_gdf[points_gdf.within(polygon)])

def check_intersection(lines_gdf, polygon):
    return int(lines_gdf.intersects(polygon).any())

def calculate_mean_raster_value(raster_path, polygon):
    with rasterio.open(raster_path) as src:
        affine = src.transform
        array = src.read(1)
    stats = zonal_stats([polygon], array, affine=affine, stats=['mean'])
    return stats[0]['mean'] if stats[0]['mean'] is not None else 0

def count_facility_types(facilities_gdf, polygon, amenity_types):
    return len(facilities_gdf[(facilities_gdf["amenity"].isin(amenity_types)) & (facilities_gdf.within(polygon))])

def count_road_types(roads_gdf, polygon, road_types):
    return len(roads_gdf[(roads_gdf["highway"].isin(road_types)) & (roads_gdf.within(polygon))])

def count_permanent_buildings(buildings_gdf, polygon):
    return len(buildings_gdf[(buildings_gdf["area_in_me"] >= 30.0) & (buildings_gdf.within(polygon))])


## Process data for each village

Iterate over the villages to extract data features.

In [10]:
village_data = []
for _, village in villages.iterrows():
    candidate_minigrids_count = count_points_in_polygon(candidate_minigrids, village.geometry)
    existing_minigrids_count = count_points_in_polygon(existing_minigrids, village.geometry)
    grid_extension_present = check_intersection(grid_extension, village.geometry)
    existing_grid_present = check_intersection(existing_grid, village.geometry)

    if grid_extension_present:
        electrification_strategy = "Grid extension"
    elif existing_grid_present:
        electrification_strategy = "Existing grid"
    elif candidate_minigrids_count > 0:
        electrification_strategy = "Candidate minigrid"
    elif existing_minigrids_count > 0:
        electrification_strategy = "Existing minigrid"
    else:
        electrification_strategy = "Solar home system"

    village_id = village["ID"]
    land_use_data = land_use[land_use["ID"] == village_id]
    protected_area_data = protected_areas[protected_areas["ID"] == village_id]


    village_info = {
        "village_id": village["ID"],
        "candidate_minigrids": candidate_minigrids_count,
        "existing_minigrids": existing_minigrids_count,
        "facilities": count_points_in_polygon(facilities, village.geometry),
        "grid_extension": grid_extension_present,
        "existing_grid": existing_grid_present,
        "mean_ndvi": calculate_mean_raster_value(ndvi_raster, village.geometry),
        "mean_wind_speed": calculate_mean_raster_value(wind_speed_raster, village.geometry),
        "mean_pvout_solar_radiation": calculate_mean_raster_value(pvout_solar_radiation_raster, village.geometry),
        "building_count": count_points_in_polygon(buildings, village.geometry),
        "permanent_building_count": count_permanent_buildings(buildings, village.geometry),
        "educational_facilities": count_facility_types(facilities, village.geometry, ["school", "college", "kindergarten"]),
        "health_facilities": count_facility_types(facilities, village.geometry, ["hospital", "HC II", "clinic"]),
        "social_facilities": count_facility_types(facilities, village.geometry, ["place_of_worship", "restaurant", "public_building"]),
        "services": count_facility_types(facilities, village.geometry, ["bank", "fuel"]),
        "primary_roads": count_road_types(roads, village.geometry, ["primary"]),
        "secondary_roads": count_road_types(roads, village.geometry, ["secondary"]),
        "tertiary_roads": count_road_types(roads, village.geometry, ["tertiary"]),
        "unclassified_roads": count_road_types(roads, village.geometry, ["unclassified"]),
        "percentage_crop_land": land_use_data["percentage_crop_land"].values[0] if not land_use_data.empty else None,
        "percentage_built_area": land_use_data["percentage_built_area"].values[0] if not land_use_data.empty else None,
        "contains_protected_area": protected_area_data["contains_protected_area"].values[0] if not protected_area_data.empty else None,
        "protected_area_name": protected_area_data["protected_area_name"].values[0] if not protected_area_data.empty else None,
        "electrification_strategy": electrification_strategy
    }
    village_data.append(village_info)

Convert to a pandas dataframe

In [11]:
df = pd.DataFrame(village_data)
df.shape

(411, 24)

In [12]:
df.head()

,village_id,candidate_minigrids,existing_minigrids,facilities,grid_extension,existing_grid,mean_ndvi,mean_wind_speed,mean_pvout_solar_radiation,building_count,...,services,primary_roads,secondary_roads,tertiary_roads,unclassified_roads,percentage_crop_land,percentage_built_area,contains_protected_area,protected_area_name,electrification_strategy
0,5500895,0,0,0,0,0,0.000000,0.000000,0.000000,7,...,0,0,0,0,0,20.356667,1.140000,False,NaN,Solar home system
1,5500896,0,0,1,0,0,0.000000,0.000000,1563.144571,17,...,0,0,0,0,1,3.387500,0.026667,True,Agoro - Agu Forest Reserve,Solar home system
2,5500897,0,0,0,0,0,0.000000,0.000000,1561.565267,99,...,0,0,0,0,0,16.712500,3.160000,True,Agoro - Agu Forest Reserve,Solar home system
3,5500898,0,0,0,0,0,0.300514,0.895073,1587.815039,101,...,0,0,0,0,0,24.385000,2.506667,True,Agoro - Agu Forest Reserve,Solar home system
4,5500899,0,0,0,0,0,0.000000,0.000000,1618.240479,0,...,0,0,0,0,0,20.292500,0.015000,True,Agoro - Agu Forest Reserve,Solar home system


Extracted features

In [16]:
df.columns

Index(['village_id', 'candidate_minigrids', 'existing_minigrids', 'facilities',
       'grid_extension', 'existing_grid', 'mean_ndvi', 'mean_wind_speed',
       'mean_pvout_solar_radiation', 'building_count',
       'permanent_building_count', 'educational_facilities',
       'health_facilities', 'social_facilities', 'services', 'primary_roads',
       'secondary_roads', 'tertiary_roads', 'unclassified_roads',
       'percentage_crop_land', 'percentage_built_area',
       'contains_protected_area', 'protected_area_name',
       'electrification_strategy'],
      dtype='object')

Save extracted data.

In [17]:
df.to_csv("village_data.csv", index=False)